In [1]:
!pip install -q transformers accelerate peft bitsandbytes datasets
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.3 MB/s eta 0:00:00


In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
print("Google Drive başarıyla bağlandı!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive başarıyla bağlandı!


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import PeftModel
from datasets import load_dataset
from transformers import BitsAndBytesConfig


base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
original_lora_path = "/content/drive/MyDrive/mistral_error_helper_lora"  # your existing LoRA adapter
new_lora_save_path = "/content/drive/MyDrive/TR_TrainingMistral/models/mistral_error_helper_lora_turkish_v2"  # new save folder

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


In [3]:
from peft import PeftModel, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

print("🔹 Loading tokenizer and base model...")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

print("🔹 Preparing model for LoRA training...")
base_model = prepare_model_for_kbit_training(base_model)  # ✅ Must come before LoRA loading

print("🔹 Loading LoRA adapter...")
model = PeftModel.from_pretrained(
    base_model,
    original_lora_path,
    is_trainable=True  # Must be set to True to continue training
)

model.train()



🔹 Loading tokenizer and base model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

🔹 Preparing model for LoRA training...
🔹 Loading LoRA adapter...


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_p

In [5]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.2.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.3.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.3.self_attn.q_proj.lora_B.default.weight
base_m

In [6]:
pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
all_data_path =  "/content/drive/MyDrive/TR_TrainingMistral/TR_Mistral_dataset.jsonl"
dataset = load_dataset("json", data_files={"data": all_data_path}, split="data")
dataset = dataset.train_test_split(test_size=0.1, seed=42)

Generating data split: 0 examples [00:00, ? examples/s]

In [5]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

def preprocess(example):
    instruction_text = (
        "Below is an error log and stack trace in English.\n"
        "Please provide a concise and clear explanation and solution in Turkish.\n"
        "Format your answer as:\n"
        "💡 Açıklama: ...\n"
        "🔧 Çözüm: ...\n\n"
        f"Error Log:\n{example['input']}"
    )
    response_text = example['output']
    full_conversation_text = f"<s>[INST] {instruction_text} [/INST] {response_text}</s>"
    tokenized = tokenizer(full_conversation_text, truncation=True, max_length=2048, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess, batched=False)


Map:   0%|          | 0/1759 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

In [6]:
import wandb

wandb.login()

# You should see a message like: "wandb: Logging into W&B as <your_username>..."
# And if successful: "wandb: Successfully logged in to Weights & BiBases!"

# 2. Then, initialize your W&B run
wandb.init(project="Mistral-Finetuning-TR", entity="nesibe-pekcakar-personal")

# wandb.log({"loss": 0.1, "accuracy": 0.9})

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nesibe-pekcakar (nesibe-pekcakar-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
from transformers import DataCollatorForLanguageModeling
tokenizer.pad_token = tokenizer.eos_token  # Padding için kullanılacak token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 🔹 Eğitim argümanları
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=new_lora_save_path,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=6,
    warmup_steps=100,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    bf16=False,
    lr_scheduler_type="cosine",
    logging_dir="./logs",
    report_to="wandb",
    run_name="mixtral_TR_V2"
)




In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

print(f"Saving new LoRA adapter to {new_lora_save_path} ...")
trainer.save_model(new_lora_save_path)

print("Training and saving complete!")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.633700,0.646674
2,0.606100,0.597721
3,0.556000,0.575246
4,0.519400,0.564964
5,0.515000,0.560147
6,0.567300,0.559371


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Saving new LoRA adapter to /content/drive/MyDrive/TR_TrainingMistral/models/mistral_error_helper_lora_turkish_v2 ...
Training and saving complete!


In [9]:
print(f"Saving new LoRA adapter to {new_lora_save_path} ...")
trainer.save_model(new_lora_save_path)

print("Training and saving complete!")

Saving new LoRA adapter to /content/drive/MyDrive/TR_TrainingMistral/models/mistral_error_helper_lora_turkish_v2 ...
Training and saving complete!
